In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
!ls
%cd gdrive/MyDrive/Datasets/Lachen/
!ls


Mounted at /content/gdrive
gdrive	sample_data
/content/gdrive/MyDrive/Datasets/Lachen
0  1  3


In [ ]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers

from sklearn.model_selection import train_test_split

# number of learnings should be until the loss converges otherwise we overfit
EPOCHS = 17
IMG_WIDTH = 32
IMG_HEIGHT = 32
NUM_CATEGORIES = 3
# number of images getting used for testing not for training in percent
TEST_SIZE = 0.2
# drop out nodes at random to make the network more robust
DROPOUT = 0.4
NORMALIZE = True
HIDDENLAYERS_SIZE = 64
STEPS_PER_EPOCH = 1
IMG_SIZE = (IMG_WIDTH, IMG_HEIGHT)
# batchsize = x -> could define one // woundn't use it except if it takes to long
# it is the amount of images in the data set which is getting looked at before an adjustment


def main():
    # Get image arrays and labels for all image files
    images, labels = load_data(os.getcwd())
    print(labels)
    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )
    # Get a compiled neural network
    model = get_model()
    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=1)

    %cd ..
    %cd ..
    !ls 
    !mkdir /content/gdrive/MyDrive/Deep_CNN_laecheln
    filename = ("/content/gdrive/MyDrive/Deep_CNN_laecheln")
    !ls
    model.save(filename)
    %cd Datasets/Lachen


def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = list()
    labels = list()

    # get currend directory
    #workingDirectory = os.getcwd()

    # add them together
    #path = workingDirectory + os.sep + data_dir
    path = data_dir
    # loop over all the directories
    for dirpath, dirnames, filenames in os.walk(path):
        print("q")
        if(filenames != ['.DS_Store']):
            # loops over all the files inside the directory
            for curFile in filenames:
                if(curFile != '.DS_Store' and curFile != 'a'):
                    file = dirpath + os.sep + curFile
                    # reads the image
                    image = cv2.imread(file)
                    # rezeises the image
                    resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT),
                                        interpolation=cv2.INTER_AREA)
                    # make the input better for the neural network
                    if(NORMALIZE):
                        resized = resized / 255.0
                    # add to pictures
                    images.append(resized)
                    # add to labels
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Lachen/3/no_smile-input-3" or 
                        dirpath == "/content/gdrive/My Drive/Datasets/Lachen/2/no_smile-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Lachen/1/no_smile-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Lachen/0/no_smile-input-0"):
                        labels.append(0)
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Lachen/1/no_face-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Lachen/2/no_face-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Lachen/3/no_face-input-3" or 
                        dirpath == "/content/gdrive/My Drive/Datasets/Lachen/0/no_face-input-0"):
                        labels.append(1)
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Lachen/3/smile-input-3" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Lachen/2/smile-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Lachen/1/smile-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Lachen/0/smile-input-0"):
                        labels.append(2)
    return(images, labels)

def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    data_augmentation = tf.keras.Sequential(
      [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                    input_shape=(IMG_HEIGHT, 
                                                                  IMG_WIDTH,
                                                                  3)),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
      ]
    )

    # decrease the rate of change towards the end
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        0.001,
        decay_steps=STEPS_PER_EPOCH*1000,
        decay_rate=1,
        staircase=False)

    INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
    # bias
    #model with several layers
    model = tf.keras.models.Sequential([
        data_augmentation,                                
        # add the convolution layer -> extract feature such as strong colour difference
        #-> lernt Kanten
        #base_model,
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu",
                               input_shape=INPUT_SHAPE),
        # use pooling -> verkleinert das Dild
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        # add another concolutional layer
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        # pool again
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        # should always increase in size or stay the same
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        # generate a form as an input layer
        tf.keras.layers.Flatten(),
        # dense all are connected -> smaller then the imput layer
        tf.keras.layers.Dense(
                HIDDENLAYERS_SIZE, kernel_regularizer=regularizers.l2(0.001),
                activation='relu'),
        # dense all are connected -> smaller then the imput layer
        tf.keras.layers.Dense(
                HIDDENLAYERS_SIZE, kernel_regularizer=regularizers.l2(0.001),
                activation='relu'),
        # add a dropout
        tf.keras.layers.Dropout(DROPOUT),
        # add the output layer -> softmax gives a probability distribution min loss = 0 max = infinity
        # continues until the end
        tf.keras.layers.Dense(NUM_CATEGORIES, "softmax")
    ])
    # compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr_schedule),
        loss="categorical_crossentropy",
        #metrics=["accuracy"]
        metrics=[tf.keras.metrics.Precision()]
    )
    return model

if __name__ == "__main__":
  main()

q
q
q
q
q
q
q
q
q
q
q
q
q
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0